# __IOC Hunting__
### By: Alejandro Perez
----------------------------


## Introduction
This Notebook will be used as an test case where we will use the ThreatFox API
to pull the most recent IOC's that have been verified and then pivot from those
to using Data from other Sources such as VirusTotal, and GreyNoise



In [1]:
from src.ThreatFoxUtils import ThreatFox
from src.VirusTotalUtils import VirusTotal
from src.GreyNoiseUtils import Greynoise
import pandas as pd
from pandas import json_normalize
import ipywidgets as widgets
from IPython.display import clear_output
import json
import time
pd.set_option('display.max_colwidth', 0)

In [2]:
# This will use the Config File in order to 
with open('config.json') as config:
    configData = json.load(config)

In [3]:
# Instantiating the VirusTotal API Key from the config file and saving it in a Variable
VT = VirusTotal(ApiKey=configData['VirusTotal Api Key'])

In [4]:
# Gathering the IOC's for the past day
IocData = ThreatFox.query_ioc_database(number_of_days=1)

[-] Gathering IOC's from past 1 days...
[+] IOC Data has been Gathered


In [5]:
# Displaying 10 random Samples from the Dataset
IocData.sample(n=10)

id                                                               ioc  \
125  9212  85192c0e836d16c56e451fee48d8d074151834f12819dd0b6e1ee2da26190e15   
151  9189  http://profile.fronthot.com/cm                                     
107  9230  3e6a71d8ee8d92d8421ea97db9c658b657c8c4428c4670c9c491a3a7125c5bc4   
144  9192  http://35.224.197.52/cm                                            
13   9326  149.91.89.121:80                                                   
117  9221  194.5.97.181:3383                                                  
70   9268  194.87.145.184:6484                                                
92   9244  185.219.134.130:80                                                 
27   9311  24.43.22.219:993                                                   
79   9258  81d913c5a441899d34ccce3583a13c07340cdb5d3b35b5ce68380b6c779d84cb   

    threat_type  \
125  payload      
151  botnet_cc    
107  payload      
144  botnet_cc    
13   botnet_cc    
117  botnet_cc    
70   botnet_cc    
92   botnet_cc    
27   botnet_cc    
79   payload      

                                                    threat_type_desc  \
125  Indicator that identifies a malware sample (payload)              
151  Indicator that identifies a botnet command&control server (C&C)   
107  Indicator that identifies a malware sample (payload)              
144  Indicator that identifies a botnet command&control server (C&C)   
13   Indicator that identifies a botnet command&control server (C&C)   
117  Indicator that identifies a botnet command&control server (C&C)   
70   Indicator that identifies a botnet command&control server (C&C)   
92   Indicator that identifies a botnet command&control server (C&C)   
27   Indicator that identifies a botnet command&control server (C&C)   
79   Indicator that identifies a malware sample (payload)              

        ioc_type  \
125  sha256_hash   
151  url           
107  sha256_hash   
144  url           
13   ip:port       
117  ip:port       
70   ip:port       
92   ip:port       
27   ip:port       
79   sha256_hash   

                                                         ioc_type_desc  \
125  SHA256 hash of a malware sample (payload)                           
151  URL that is used for botnet Command&control (C&C)                   
107  SHA256 hash of a malware sample (payload)                           
144  URL that is used for botnet Command&control (C&C)                   
13   ip:port combination that is used for botnet Command&control (C&C)   
117  ip:port combination that is used for botnet Command&control (C&C)   
70   ip:port combination that is used for botnet Command&control (C&C)   
92   ip:port combination that is used for botnet Command&control (C&C)   
27   ip:port combination that is used for botnet Command&control (C&C)   
79   SHA256 hash of a malware sample (payload)                           

                 malware malware_printable                   malware_alias  \
125  win.agent_tesla      Agent Tesla       AgenTesla,AgentTesla,Negasteal   
151  win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike    
107  win.netwire          NetWire RC        NetWeird,NetWire,Recam           
144  win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike    
13   win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike    
117  win.netwire          NetWire RC        NetWeird,NetWire,Recam           
70   win.redline_stealer  RedLine Stealer   None                             
92   win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike    
27   win.qakbot           QakBot            Pinkslipbot,Qbot,Quakbot         
79   win.agent_tesla      Agent Tesla       AgenTesla,AgentTesla,Negasteal   

                                                         malware_malpedia  \
125  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla       
151  https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike     
107  htt

## __GreyNoise__
------------
In this Section, We will use Greynoise which a Service that will return information on IP Addresses in order to rule out any Whitenoise when it comes to scanning. Sources like Shodan, and Censys scan the Internet regularly but mean no harm when doing so, but this service will assist us in finding the IP Addresses that could be scanning maliciously, if there are any in the current dataset.

In order to do this we will take the Dataset from ThreatFox, Filter for IP Addresses, and then Apply it and have a Dataframe returned to us if there are any malicious IP Addresses to track.

In [6]:
# Filtering the Data Set for solely IP:Port Entries
Suspicious_IP = IocData[IocData['ioc_type']=='ip:port']

# Displaying the newly filtered Dataframe (only first 5 rows)
display(Suspicious_IP.head(5))

id                  ioc threat_type  \
1  9337  185.140.53.54:6687   botnet_cc    
2  9336  172.111.168.19:6381  botnet_cc    
3  9335  5.101.66.180:80      botnet_cc    
5  9333  185.162.235.111:80   botnet_cc    
6  9331  174.138.0.82:80      botnet_cc    

                                                  threat_type_desc ioc_type  \
1  Indicator that identifies a botnet command&control server (C&C)  ip:port   
2  Indicator that identifies a botnet command&control server (C&C)  ip:port   
3  Indicator that identifies a botnet command&control server (C&C)  ip:port   
5  Indicator that identifies a botnet command&control server (C&C)  ip:port   
6  Indicator that identifies a botnet command&control server (C&C)  ip:port   

                                                       ioc_type_desc  \
1  ip:port combination that is used for botnet Command&control (C&C)   
2  ip:port combination that is used for botnet Command&control (C&C)   
3  ip:port combination that is used for botnet Command&control (C&C)   
5  ip:port combination that is used for botnet Command&control (C&C)   
6  ip:port combination that is used for botnet Command&control (C&C)   

               malware malware_printable                  malware_alias  \
1  win.nanocore         Nanocore RAT      Nancrat,NanoCore                
2  win.asyncrat         AsyncRAT          None                            
3  win.redline_stealer  RedLine Stealer   None                            
5  win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike   
6  win.cobalt_strike    Cobalt Strike     Agentemis,BEACON,CobaltStrike   

                                                       malware_malpedia  \
1  https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore          
2  https://malpedia.caad.fkie.fraunhofer.de/details/win.asyncrat          
3  https://malpedia.caad.fkie.fraunhofer.de/details/win.redline_stealer   
5  https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike     
6  https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike     

   confidence_level               first_seen last_seen  \
1  100               2021-04-21 02:45:27 UTC  None       
2  100               2021-04-21 01:55:25 UTC  None       
3  100               2021-04-21 00:55:57 UTC  None       
5  50                2021-04-20 22:20:04 UTC  None       
6  50                2021-04-20 22:20:03 UTC  None       

                             reference        reporter              tags  
1  None                                 abuse_ch        [NanoCore, RAT]   
2  None                                 abuse_ch        [asyncrat, RAT]   
3  None                                 abuse_ch        [RedLineStealer]  
5  https://twitter.com/MichalKoczwara/  MichalKoczwara  [Cobalt Strike]   
6  https://twitter.com/MichalKoczwara/  MichalKoczwara  [Cobalt Strike]

In [7]:
# Creating List object to store GreyNoise data
GreyNoise_Data_List  = []

# Iterating through the Suspicious_IP Data
print("[-] Scanning IP's...")
for IP_Port in IocData['ioc']:
    # Splitting the IP:Port Combination for solely the IP Address
    IP = IP_Port.split(":")[0]
    # Using our Custom Function for GreyNoise to return the Information to us
    GreyNoise_Scan = Greynoise.lookup_IP(IP)
    # Checks to catch any errors such as this one and will pass over it
    if GreyNoise_Scan['message'] == "Request is not a valid routable IPv4 address":
        pass
    else:
        # Appending the Data to the list we made above
        GreyNoise_Data_List.append(GreyNoise_Scan)

print("[+] Scan has completed")


[-] Scanning IP's...
[+] Scan has completed


In [8]:
# Takes the List of Data and flattens the Json into a Dataframe
GreyNoiseDataframe = json_normalize(GreyNoise_Data_List)
# Displays the Dataframe
GreyNoiseDataframe

ip  noise   riot  \
0   185.140.53.54    False  False   
1   172.111.168.19   False  False   
2   5.101.66.180     False  False   
3   185.162.235.111  True   False   
4   174.138.0.82     False  False   
5   94.103.94.203    True   False   
6   185.150.119.33   False  False   
7   185.150.119.33   False  False   
8   5.199.162.115    False  False   
9   5.199.162.115    False  False   
10  5.199.162.116    False  False   
11  149.91.89.121    False  False   
12  78.128.112.136   False  False   
13  78.128.112.137   False  False   
14  37.61.205.212    False  False   
15  37.61.205.212    False  False   
16  95.217.124.100   False  False   
17  205.185.119.191  True   False   
18  45.93.201.181    False  False   
19  195.123.208.194  False  False   
20  24.43.22.219     False  False   
21  192.169.69.25    False  False   
22  45.133.1.59      False  False   
23  31.148.99.134    False  False   
24  185.38.142.241   False  False   
25  194.87.145.184   False  False   
26  51.178.146.147   False  False   
27  45.67.228.147    False  False   
28  45.90.46.164     False  False   
29  45.15.143.209    False  False   
30  135.125.166.131  False  False   
31  79.134.225.49    False  False   
32  185.219.134.130  False  False   
33  27.102.130.117   False  False   
34  63.32.110.179    False  False   
35  46.183.220.67    False  False   
36  46.183.220.67    False  False   
37  104.236.223.230  False  False   
38  152.89.247.26    False  False   
39  147.124.218.73   False  False   
40  45.85.90.235     False  False   
41  194.5.97.181     False  False   
42  79.134.225.50    False  False   
43  194.5.97.183     False  False   
44  37.59.109.58     False  False   
45  54.174.145.85    False  False   

                                                                 message  \
0   IP not observed scanning the internet or contained in RIOT data set.   
1   IP not observed scanning the internet or contained in RIOT data set.   
2   IP not observed scanning the internet or contained in RIOT data set.   
3   Success                                                                
4   IP not observed scanning the internet or contained in RIOT data set.   
5   Success                                                                
6   IP not observed scanning the internet or contained in RIOT data set.   
7   IP not observed scanning the internet or contained in RIOT data set.   
8   IP not observed scanning the internet or contained in RIOT data set.   
9   IP not observed scanning the internet or contained in RIOT data set.   
10  IP not observed scanning the internet or contained in RIOT data set.   
11  IP not observed scanning the internet or contained in RIOT data set.   
12  IP not observed scanning the internet or contained in RIOT data set.   
13  IP not observed scanning the internet or contained in RIOT data set.   
14  IP not observed scanning the internet or contained in RIOT data set.   
15  IP not observed scanning the internet or contained in RIOT data set.   
16  IP not observed scanning the internet or contained in RIOT data set.   
17  Success                                                                
18  IP not observed scanning the internet or contained in RIOT data set.   
19  IP not observed scanning the internet or contained in RIOT data set.   
20  IP not observed scanning the internet or contained in RIOT data set.   
21  IP not observed scanning the internet or contained in RIOT data set.   
22  IP not observed scanning the internet or contained in RIOT data set.   
23  IP not observed scanning the internet or contained in RIOT data set.   
24  IP not observed scanning the internet or contained in RIOT data set.   
25  IP not observed scanning the internet or contained in RIOT data set.   
26  IP not observed scanning the internet or contained in RIOT data set.   
27  IP not observed scanning the internet or contained in RIOT data set.   
28  IP not observed scanning the internet or contained in RIOT data set. 

## __VirusTotal__
----------------
This Section is Dedicated to grabbing the Hashes from ThreatFox and pull reports from VirusTotal on them. In order to do this, we will follow the same methodology and filter out the hashes from our ThreatFox Dataset and then begin to pull information from VirusTotal.

However this section will be interactive with a small Widget tool to search for reports on the hash

In [9]:
# Filters for Suspicious Hashes
Suspicious_Hashes = IocData[IocData['ioc_type']=='sha256_hash']

In [10]:
# Displays the Suspicious Hashes first 5 rows
Suspicious_Hashes.head(5)

id                                                               ioc  \
28  9307  c50583bdebf9d38a325a1bb74e769cca7fb9fee7722aedb360b8fec069775f52   
29  9308  b8ff56a3f38384dcd38e3eee204d84a046364226151687fba057c75fb987c15d   
30  9309  e20251932fbb15ab76bded960ee7f6a4dc33847603e1a410f213636df14eba58   
31  9310  4d5efc07157713342c2b04e645ad4df03a689c09f92c928024518ebffa883bfd   
33  9302  67e1c4b84161a155a5b9a8afcf7e62f357f6f5d12576160d4f0559d86f18173a   

   threat_type                                      threat_type_desc  \
28  payload     Indicator that identifies a malware sample (payload)   
29  payload     Indicator that identifies a malware sample (payload)   
30  payload     Indicator that identifies a malware sample (payload)   
31  payload     Indicator that identifies a malware sample (payload)   
33  payload     Indicator that identifies a malware sample (payload)   

       ioc_type                              ioc_type_desc          malware  \
28  sha256_hash  SHA256 hash of a malware sample (payload)  win.agent_tesla   
29  sha256_hash  SHA256 hash of a malware sample (payload)  win.agent_tesla   
30  sha256_hash  SHA256 hash of a malware sample (payload)  win.agent_tesla   
31  sha256_hash  SHA256 hash of a malware sample (payload)  win.agent_tesla   
33  sha256_hash  SHA256 hash of a malware sample (payload)  win.agent_tesla   

   malware_printable                   malware_alias  \
28  Agent Tesla       AgenTesla,AgentTesla,Negasteal   
29  Agent Tesla       AgenTesla,AgentTesla,Negasteal   
30  Agent Tesla       AgenTesla,AgentTesla,Negasteal   
31  Agent Tesla       AgenTesla,AgentTesla,Negasteal   
33  Agent Tesla       AgenTesla,AgentTesla,Negasteal   

                                                    malware_malpedia  \
28  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla   
29  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla   
30  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla   
31  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla   
33  https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla   

    confidence_level               first_seen last_seen  \
28  50                2021-04-20 18:03:50 UTC  None       
29  50                2021-04-20 18:03:50 UTC  None       
30  50                2021-04-20 18:03:50 UTC  None       
31  50                2021-04-20 18:03:50 UTC  None       
33  50                2021-04-20 17:03:45 UTC  None       

                                                      reference    reporter  \
28  https://twitter.com/RedBeardIOCs/status/1384568479035142144  Virus_Deck   
29  https://twitter.com/RedBeardIOCs/status/1384568479035142144  Virus_Deck   
30  https://twitter.com/RedBeardIOCs/status/1384568479035142144  Virus_Deck   
31  https://twitter.com/RedBeardIOCs/status/1384568479035142144  Virus_Deck   
33  https://twitter.com/RedBeardIOCs/status/1384553357747146753  Virus_Deck   

            tags  
28  [AgentTesla]  
29  [AgentTesla]  
30  [AgentTesla]  
31  [AgentTesla]  
33  [AgentTesla]

In [11]:
# Building the Output of the Widgets
VT_File_Report_Output = widgets.Output()

# Building the Hash Selection Dropdown Menu
VT_Hash_Selection = widgets.Dropdown(
    options = Suspicious_Hashes['ioc'].tolist(),
    description = 'Hashes:'
)

# Building the Button for the VT Hash Search
VT_SearchButton = widgets.Button(
    description = 'Search'
)

# adding action to the VT Search Button
def VT_Search_Action(x):
    with VT_File_Report_Output:
        clear_output()
        # Runs the Report Search for the Hash
        Report = VT.get_file_information(FileHash=VT_Hash_Selection.value)
        
        # Checks if what is returned back is a Dataframe,
        # which will return a Transposed Dataframe for easier readability
        if isinstance(Report, pd.DataFrame):
            display(Report.T)
        
        # If it is not a Dataframe, There was an error in finding the
        # Hash and so it will return a String error
        else:
            print(Report)

            
# Attaching the Action Function to the button
VT_SearchButton.on_click(VT_Search_Action)

# Creating a Vertical Box to handle the widgets
widgets.VBox([VT_Hash_Selection, VT_SearchButton, VT_File_Report_Output])